In [1]:
#page to be webscraped
url = 'https://oldschool.runescape.wiki/w/Zulrah'

In [2]:
#libraries needed
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
#access the contents of the web
response = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})

#extract HTML from response
page_content = response.content

#parse HTML
soup = BeautifulSoup(page_content,'lxml')

In [10]:
#find all tables in HTML
my_tables = soup.find_all('table', attrs = {'class':'wikitable sortable filterable item-drops autosort=4,a'})
print(my_tables)

[<table class="wikitable sortable filterable item-drops autosort=4,a" style="text-align:center">
<tbody><tr>
<th class="drops-img-header drop-disp-btn unsortable" style="border-right: none;" title="Sort items by category, rarity and name."></th>
<th class="item-col" style="border-left: none;">Item</th>
<th data-sort-type="number">Quantity</th>
<th>Rarity</th>
<th class="ge-column" data-sort-type="number">Price</th>
<th class="alch-column" data-sort-type="number">High Alch</th></tr>
<tr style="text-align:center"><td class="inventory-image"><span class="mw-default-size" typeof="mw:File"><a href="/w/Zulrah%27s_scales" title="Zulrah's scales"><img alt="Zulrah's scales.png: Zulrah drops Zulrah's scales with rarity Always in quantity 100-299" class="mw-file-element" data-file-height="29" data-file-width="30" decoding="async" height="29" loading="lazy" src="/images/Zulrah%27s_scales_5.png?93aed" width="30"/></a></span></td><td class="item-col" style="text-align:left"><a href="/w/Zulrah%27s_sc

In [21]:
#initialize empty dictionary to see all tables in DF
loot_tables = {}
table_index = 0

#obtain data into DF
for table in my_tables:
    #parse out table rows
    table_rows = table.find_all('tr')

    #parse out table header
    table_header = table.find_all('th')

    #parse columns from table header
    cols = []
    for item in table_header:
        if len(item.text) > 0:
            cols.append(item.text)

    #parse data from each row
    data = []
    
    for row in table_rows:
        table_data = row.find_all('td')
        
        row_data = [item.text for item in table_data if item.text != '']

        if len(row_data) > 0:
            data.append(row_data)

    #create DF
    output_frame = pd.DataFrame(data, columns = cols)

    loot_tables.setdefault(table_index, output_frame)
    table_index +=1
    
output_frame #will only show the last table 
    #print(data)
    #print(cols)


In [22]:
loot_tables[0] #check first table

,Item,Quantity,Rarity,Price,High Alch
0,Zulrah's scales,100–299,Always,"27,900–83,421","1,200–3,588"


In [23]:
loot_tables[1] #check second table

,Item,Quantity,Rarity,Price,High Alch
0,Tanzanite fang,1,"2 × 1/1,024","5,685,017","66,000"
1,Magic fang,1,"2 × 1/1,024","5,596,050","61,200"
2,Serpentine visage,1,"2 × 1/1,024","5,553,376","63,000"
3,Uncut onyx,1,"2 × 1/1,024","2,449,506","120,000"


In [25]:
#put all the tables into one whole DF since all the tables have the same column names and numebr of columns
loot_table = pd.concat(loot_tables.values())
loot_table.set_index('Item', inplace = True)
loot_table

,Quantity,Rarity,Price,High Alch
Item,,,,
Zulrah's scales,100–299,Always,"27,900–83,421","1,200–3,588"
Tanzanite fang,1,"2 × 1/1,024","5,685,017","66,000"
Magic fang,1,"2 × 1/1,024","5,596,050","61,200"
Serpentine visage,1,"2 × 1/1,024","5,553,376","63,000"
Uncut onyx,1,"2 × 1/1,024","2,449,506","120,000"
...,...,...,...,...
Rune javelin,5,"2 × 1/22,573.51; 1/11,463.11[d 2]",950,"1,200"
Brimstone key,1,1/50[d 1],Not sold,N/A
Clue scroll (elite),1,1/75[d 2][4],Not sold,12


In [ ]:
#Part 3: Create Simple Django website to host predictions